# More cleaning!

In [1]:
import dtale 
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

In [2]:
df = pd.read_json('../data/model_data/data.json')

In [3]:
# drop all entries with no listed price :(
df = df[df['price'].notnull()]

We want to log-transform price. We do this for interpretability reasons; see the section at the end about RMSE.

In [4]:
df.price = df.price.apply(lambda x: np.log(x))

Here, we will DROP unneeded columns/entries that will make regression very difficult while also not adding a whole lot; this includes the categorical variables, very sparse fields like `year_built`, and redundant information like `address`.

In [5]:
df = df[df['property_type'].isin(['house', 'apartment'])].reset_index()
df = df.drop(['images', 'description', 'company', 'address', 'year_built', 'utilities', 'index', 'location_data', 'raw_data', 'updated_addresses'], axis=1)

# sanity check
df = df[df['distance_to_CCTC'] < 20]

DTale is a cool tool to visualize dataframes and make plots on the spot!  

In [6]:
# dtale.show(df)

Here we are "imputing" missing values. There are definitely better ways to do this, but in the interest of time, we will let the regression models handle it themselves if possible.

In [7]:
def fix_bed(x):
    try:
        x = float(x)
        if x == 0:
            return 0
        return x
    
    except:
        if x == 'room':
            return 1
        return np.nan

df.bed = df.bed.apply(fix_bed).astype(float)

In [8]:
def clean_area(area):
    if isinstance(area, int):
        return float(area)
    elif isinstance(area, float):
        return area
    elif re.search('(\d+)(?:\s*[\-\/+*]\s*)(\d+)', str(area)) != None and len(area) > 4:
        var = re.search('(\d+)(?:\s*[\-\/+*]\s*)(\d+)', str(area))
        return (float(var.group(1)) + float(var.group(2))) / 2
    elif area and (re.search('(\d+)', area) != None):
        if re.search('(\d+)(?:x)(\d+)', area) != None:
            num = re.search('(\d+)(?:x)(\d+)', area)
            return float(num.group(1)) * float(num.group(2))
        area = area.replace(',', '').replace('~', '')
        return re.search('(\d+)', area).group(1)
    else:
        return np.nan
    
df['area'] = df['area'].apply(clean_area).astype(float).replace(0, np.nan)

In [9]:
# new_area = average of all listings with same number of bedrooms, if possible
def impute_area(x):
    bed = x['bed']
    area = x['area']
    
    try:
        bed = float(bed)
        if np.isnan(area):
            x['area'] = np.mean(df[(df['bed'] == bed) & df['area'].notnull()].area)
            return x
        return x
    
    except:
        x['area'] = np.nan
        return x

df = df.apply(impute_area, axis=1)

In [10]:
def impute_laundry(x):
    try:
        x = float(x)
        return x
    
    except:
        return np.nan

df.laundry = df.laundry.apply(impute_laundry)

In [11]:
def impute_pets(x):
    try:
        x = float(x)
        return x
    
    except:
        return np.nan

df.pets = df.pets.apply(impute_pets)

In [12]:
def impute_parking(x):
    try:
        x = float(x)
        return x
    
    except:
        return np.nan

df.parking = df.parking.apply(impute_parking)

Here, we'll split off price to use as our labels during the training process later. We also have to modify `X` so that everything is scaled properly.

In [16]:
# split data into features (X) and labels (y)
X, y = df.drop('price', axis=1), df['price']

In [17]:
# sanity check - pre-normalization
X

,bed,bath,area,neighborhood,laundry,pets,parking,property_type,lat,lon,distance_to_CCTC,distance_to_Pierpont,distance_to_Stadium,distance_to_Union,distance_to_Ross,distance_to_IM,distance_to_NCRB,distance_to_CCRB,distance_to_UgLi
0,2.0,1.0,876.0,Other Surrounding Areas,1.0,1.0,1.0,apartment,42.238978,-83.718505,4.557801,5.802060,3.886862,4.448382,4.637215,4.213861,6.254798,4.846131,4.358015
1,4.0,2.5,2000.0,Other Surrounding Areas,1.0,1.0,1.0,house,42.262519,-83.738165,1.762333,3.614058,0.942182,1.434291,1.726546,1.177845,4.113428,2.100388,1.462828
2,1.0,1.0,800.0,Other Surrounding Areas,1.0,1.0,1.0,apartment,42.293951,-83.683362,4.577147,2.815139,6.215570,5.219680,4.846902,6.056612,2.575089,4.372745,4.872060
3,1.0,1.0,800.0,Other Surrounding Areas,1.0,1.0,1.0,apartment,42.293951,-83.683362,4.577147,2.815139,6.215570,5.219680,4.846902,6.056612,2.575089,4.372745,4.872060
4,1.0,1.0,800.0,Other Surrounding Areas,1.0,1.0,0.0,apartment,42.293951,-83.683362,4.577147,2.815139,6.215570,5.219680,4.846902,6.056612,2.575089,4.372745,4.872060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5653,10.0,4.0,NaN,East Packard,1.0,1.0,1.0,house,42.270830,-83.739093,0.890524,2.880573,0.963084,0.522900,0.805617,0.836366,3.371632,1.228505,0.559146
5654,9.0,5.0,NaN,East Packard,1.0,1.0,1.0,house,42.270071,-83.734811,0.900015,2.747953,1.233673,0.791724,0.930082,1.175646,3.246113,1.230819,0.650585
5655,9.0,5.0,NaN,East Packard,1.0,1.0,1.0,house,42.270071,-83.734811,0.900015,2.747953,1.233673,0.791724,0.930082,1.175646,3.246113,1.230819,0.650585
5656,9.0,5.0,NaN,East Packard,1.0,1.0,1.0,house,42.270071,-83.734811,0.900015,2.747953,1.233673,0.791724,0.930082,1.175646,3.246113,1.230819,0.650585


In [18]:
# min-max normalize numeric features
X_float = X.select_dtypes(include=np.number)
col = X_float.columns.tolist()
X_float = MinMaxScaler().fit_transform(X_float)
X_other = X.select_dtypes(exclude=np.number)
X = pd.concat([pd.DataFrame(X_float, columns=col), X_other.reset_index().drop('index', axis=1)], axis=1)

Note that it should be ok to normalize the coordinates as well, since we are dealing with such a small region that the Earth's curvature is negligible and can therefore be ignored.

In [19]:
# sanity check - post-normalization
data = pd.concat([y.reset_index().drop('index', axis=1), X], axis=1)
data

,price,bed,bath,area,laundry,pets,parking,lat,lon,distance_to_CCTC,distance_to_Pierpont,distance_to_Stadium,distance_to_Union,distance_to_Ross,distance_to_IM,distance_to_NCRB,distance_to_CCRB,distance_to_UgLi,neighborhood,property_type
0,6.998510,0.2,0.111111,0.102953,1.0,1.0,1.0,0.277344,0.556060,0.225814,0.259883,0.186173,0.222507,0.227654,0.211189,0.279171,0.243785,0.217891,Other Surrounding Areas,apartment
1,7.600902,0.4,0.277778,0.235204,1.0,1.0,1.0,0.372139,0.512543,0.082072,0.152201,0.034829,0.066600,0.074705,0.058171,0.174699,0.103561,0.068031,Other Surrounding Areas,house
2,7.233455,0.1,0.111111,0.094011,1.0,1.0,1.0,0.498713,0.633849,0.226808,0.112882,0.305859,0.262403,0.238672,0.304066,0.099647,0.219609,0.244499,Other Surrounding Areas,apartment
3,7.077498,0.1,0.111111,0.094011,1.0,1.0,1.0,0.498713,0.633849,0.226808,0.112882,0.305859,0.262403,0.238672,0.304066,0.099647,0.219609,0.244499,Other Surrounding Areas,apartment
4,7.226209,0.1,0.111111,0.094011,1.0,1.0,0.0,0.498713,0.633849,0.226808,0.112882,0.305859,0.262403,0.238672,0.304066,0.099647,0.219609,0.244499,Other Surrounding Areas,apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,6.495266,1.0,0.444444,NaN,1.0,1.0,1.0,0.405607,0.510488,0.037244,0.116102,0.035904,0.019458,0.026312,0.040960,0.138508,0.059034,0.021255,East Packard,house
4767,6.684612,0.9,0.555556,NaN,1.0,1.0,1.0,0.402552,0.519969,0.037732,0.109575,0.049811,0.033363,0.032853,0.058060,0.132385,0.059152,0.025988,East Packard,house
4768,6.684612,0.9,0.555556,NaN,1.0,1.0,1.0,0.402552,0.519969,0.037732,0.109575,0.049811,0.033363,0.032853,0.058060,0.132385,0.059152,0.025988,East Packard,house
4769,6.684612,0.9,0.555556,NaN,1.0,1.0,1.0,0.402552,0.519969,0.037732,0.109575,0.049811,0.033363,0.032853,0.058060,0.132385,0.059152,0.025988,East Packard,house


**Check that everything has been done.** Then, save the data to the repository.

In [20]:
# save data to repository
X.to_json('../data/model_data/X.json')
y.to_json('../data/model_data/y.json')

In [21]:
print('number of entries:', X.shape[0])

number of entries: 4771
